In [2]:
import pandas as pd
import numpy as np
import re
import gensim
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from wordcloud import WordCloud
from gensim import corpora, models, similarities
import pyLDAvis
from pprint import pprint
import pickle 
import os
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
import random
from gensim.models.ldamodel import LdaModel
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import MmCorpus
import mymodule

In [3]:
data = pd.read_csv('dataset/DatasetLegal.csv')
str_answer = data['answer'].astype(str)
str_answer = str_answer.map(lambda x: re.sub('[,.!?#/]', '', x))
str_question = data['question'].astype(str)
str_question = str_question.map(lambda x: re.sub('[,.!?*/]', '', x))

In [4]:
sentense_token = []
for i in range(len(str_answer)):
  sentense_token.append(str_question[i])
  sentense_token.append(str_answer[i])

train_data = sentense_token[:13062]
test_data = sentense_token[13062:]

In [5]:
word_token_answer = []
for sentense in train_data:
  word = word_tokenize(sentense, engine='newmm')
  word_token_answer.append(word)

In [6]:
stopwords = list(thai_stopwords())
read_stopwords = pd.read_csv('dataset/add_stopwords.csv')
add_stopwords = read_stopwords['stopword'].values.tolist()
processed_answer = []
for sentense in word_token_answer:
  each_sentense = []
  for word in sentense:
    if(word not in stopwords + add_stopwords):
      each_sentense.append(word)
  processed_answer.append(each_sentense)
print(processed_answer[0][:8])

['มีเรื่อง', 'ปรึกษา', 'ครอบครัว', 'สินสมรส', 'สินส่วนตัว', 'การทราบ', 'ข้อเท็จจริง', 'วิธีการ']


In [7]:
id2word = corpora.Dictionary(processed_answer)
# print(id2word)
with open('model/id2word.pkl', 'wb') as f:
    pickle.dump(id2word, f)

2023-04-20 10:33:36.496 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary<0 unique tokens: []>
2023-04-20 10:33:37.455 INFO    gensim.corpora.dictionary: adding document #10000 to Dictionary<14628 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...>
2023-04-20 10:33:37.763 INFO    gensim.corpora.dictionary: built Dictionary<16502 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...> from 13062 documents (total 654776 corpus positions)
2023-04-20 10:33:37.764 INFO    gensim.utils: Dictionary lifecycle event {'msg': "built Dictionary<16502 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...> from 13062 documents (total 654776 corpus positions)", 'datetime': '2023-04-20T10:33:37.764090', 'gensim': '4.3.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [8]:
corpus = []
for text in processed_answer:
  vec = id2word.doc2bow(text)
  corpus.append(vec)

In [9]:
num_topics = 6
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=100,
                                       chunksize=2000,
                                       passes=10,
                                       alpha=0.9,
                                       eta=0.5
                                       )

2023-04-20 10:33:38.612 INFO    gensim.models.ldamodel: using serial LDA version on this node
2023-04-20 10:33:38.649 INFO    gensim.models.ldamodel: running online (multi-pass) LDA training, 6 topics, 10 passes over the supplied corpus of 13062 documents, updating model once every 2000 documents, evaluating perplexity every 13062 documents, iterating 100x with a convergence threshold of 0.001000
2023-04-20 10:33:38.650 INFO    gensim.models.ldamodel: PROGRESS: pass 0, at document #2000/13062
2023-04-20 10:33:44.824 INFO    gensim.models.ldamodel: merging changes from 2000 documents into a model of 13062 documents
2023-04-20 10:33:44.858 INFO    gensim.models.ldamodel: topic #4 (0.900): 0.013*"เงิน" + 0.009*"ฟ้อง" + 0.009*"ศาล" + 0.008*"จ่าย" + 0.007*"ที่ดิน" + 0.007*"โอน" + 0.007*"บริษัท" + 0.006*"สิทธิ" + 0.006*"มาตรา" + 0.006*"บาท"
2023-04-20 10:33:44.860 INFO    gensim.models.ldamodel: topic #2 (0.900): 0.010*"สามี" + 0.010*"มาตรา" + 0.008*"ศาล" + 0.008*"ฟ้อง" + 0.007*"กรณี" + 0.00

In [10]:
with open('model/lda_model.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

In [11]:
lda_model.show_topics(num_topics=6, num_words=10)

[(0,
  '0.049*"ที่ดิน" + 0.032*"บ้าน" + 0.023*"พ่อ" + 0.021*"แม่" + 0.020*"โอน" + 0.013*"ชื่อ" + 0.013*"ขาย" + 0.012*"แบ่ง" + 0.012*"มรดก" + 0.011*"ผู้จัดการมรดก"'),
 (1,
  '0.014*"ตำรวจ" + 0.014*"แฟน" + 0.014*"โดน" + 0.010*"รถ" + 0.009*"เพื่อน" + 0.008*"ทราบ" + 0.008*"แจ้งความ" + 0.008*"ถาม" + 0.007*"ทนาย" + 0.007*"บ้าน"'),
 (2,
  '0.056*"สามี" + 0.042*"บุตร" + 0.023*"ภรรยา" + 0.020*"บิดา" + 0.020*"ลูก" + 0.018*"เด็ก" + 0.015*"มาตรา" + 0.014*"มารดา" + 0.013*"ภริยา" + 0.013*"หญิง"'),
 (3,
  '0.035*"ศาล" + 0.021*"มาตรา" + 0.015*"ฟ้อง" + 0.014*"จำเลย" + 0.012*"เจ้าหนี้" + 0.012*"ชำระหนี้" + 0.010*"ลูกหนี้" + 0.009*"โจทก์" + 0.009*"ความผิด" + 0.009*"บังคับ"'),
 (4,
  '0.028*"เงิน" + 0.025*"บริษัท" + 0.020*"จ่าย" + 0.017*"บาท" + 0.015*"สัญญา" + 0.012*"รถ" + 0.011*"แจ้ง" + 0.010*"ค่า" + 0.010*"ธนาคาร" + 0.009*"วันที่"'),
 (5,
  '0.042*"นายจ้าง" + 0.035*"ลูกจ้าง" + 0.024*"สิทธิ" + 0.022*"มาตรา" + 0.015*"เลิกจ้าง" + 0.015*"ทำงาน" + 0.014*"จ่าย" + 0.014*"กรณี" + 0.011*"ค่าจ้าง" + 0.010*"คุ้มคร

In [2]:
import pickle
topic_dict = {
    0 : "succession",
    1 : "violation",
    2 : "family",
    3 : "criminal",
    4 : "contract",
    5 : "labor",
}

with open('model/topic_dict.pkl', 'wb') as f:
    pickle.dump(topic_dict, f)


In [ ]:
import pyLDAvis.gensim_models as gensimvis
num_topics = 6
with open('model/id2word.pkl', 'rb') as f:
  id2word = pickle.load(f)
with open('model/lda_model.pkl', 'rb') as f:
  lda_model = pickle.load(f)
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

LDAvis_data_filepath
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

The sections below working on lda model in question part

In [13]:
sentense_token_question = []
# for each_question in str_question:
#   sentense_token_question.append(each_question)
for i in range(6531):
  sentense_token_question.append(str_question[i])
# print(len(question_data))
topic_question = []
corpus_question = []
for sentense in sentense_token_question:
  processed_question = mymodule.preprocess(sentense)
  corpus_question.append(processed_question)
  topic = lda_model.get_document_topics(processed_question)
  topic_question.append(topic)

In [14]:
with open('model/question_lda.pkl', 'wb') as f:
    pickle.dump(topic_question, f)
MmCorpus.serialize('model/corpus_question.mm', corpus_question)

2023-04-20 10:41:08.386 INFO    gensim.corpora.mmcorpus: storing corpus in Matrix Market format to model/corpus_question.mm
2023-04-20 10:41:08.397 INFO    gensim.matutils: saving sparse matrix to model/corpus_question.mm
2023-04-20 10:41:08.398 INFO    gensim.matutils: PROGRESS: saving document #0
2023-04-20 10:41:08.551 INFO    gensim.matutils: PROGRESS: saving document #1000
2023-04-20 10:41:08.701 INFO    gensim.matutils: PROGRESS: saving document #2000
2023-04-20 10:41:08.841 INFO    gensim.matutils: PROGRESS: saving document #3000
2023-04-20 10:41:09.023 INFO    gensim.matutils: PROGRESS: saving document #4000
2023-04-20 10:41:09.186 INFO    gensim.matutils: PROGRESS: saving document #5000
2023-04-20 10:41:09.387 INFO    gensim.matutils: PROGRESS: saving document #6000
2023-04-20 10:41:09.490 INFO    gensim.matutils: saved 6531x16502 matrix, density=0.217% (233867/107774562)
2023-04-20 10:41:09.494 INFO    gensim.corpora.indexedcorpus: saving MmCorpus index to model/corpus_questi

The sections below are in the process of being updated with new documents.

In [49]:
new_doc = random.choice(test_data)
print(new_doc)
test_doc = mymodule.preprocess(new_doc)

อยากทราบว่าดิฉันได้ทำงานแล้วได้ยักยอกทรัพย์ของบริษัทกับเพื่อนไปประมาณสองแสนบาทถ้าทางบริษัทจะเอาก่อนเป็นก้อนแล้วที่เหลือให้เราผ่อนชำระถ้าเราไม่มีให้เป็นก้อนแต่เราให้มารดาไปประณีประณอมหนี้โดยการผ่อนชำระจะได้มั้ยถ้าเขาไม่ยอมแล้วแจ้งความตำรวจจะทำยังไงบ้างรบกวนตอบคำถามด้วยนะค


In [50]:
with open('model/question_lda.pkl', 'rb') as f:
    topic_question = pickle.load(f)
with open('model/topic_dict.pkl', 'rb') as f:
    topic_dict = pickle.load(f)
new_doc_topics = lda_model.get_document_topics(test_doc)
new_doc_topics_dict = {topic_dict[topic]: prob for topic, prob in new_doc_topics}
print(new_doc_topics_dict)

{'Personal Right': 0.19088428, 'Labor': 0.11647532, 'Contract': 0.50725573, 'Sentence': 0.070186555, 'Succession': 0.047750182, 'Family': 0.06744795}


In [24]:
# print(index)
# print(corpus_lda)
# print(sims)

MatrixSimilarity<7031 docs, 16502 features>
[0.41053373 0.14165123 0.9670263  ... 0.17920232 0.33461368 0.55741996]


In [51]:
corpus = corpus_question
data = pd.read_csv('dataset/DatasetLegal.csv')
with open('model/id2word.pkl', 'rb') as f:
  id2word = pickle.load(f)
corpus_lda = lda_model[corpus]
index = similarities.MatrixSimilarity(corpus_lda, num_features=len(id2word))
sims = index[new_doc_topics]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(f"Topic distribution for new document : {new_doc_topics}\n{new_doc}\n")
for doc_id, similarity in sims_sorted[:5]:
  print(f"Document ID: {doc_id}, Similarity score: {similarity}")
  print(data.answer[doc_id])
  print("Topic distribution for similar document : ")
  for num, dis in corpus_lda[doc_id]:
    print(f"\t({topic_dict.get(num)}, {'%.5f' %dis})")

2023-04-17 00:20:49.334 INFO    gensim.similarities.docsim: creating matrix with 6531 documents and 16502 features


Topic distribution for new document : [(0, 0.19088428), (1, 0.11647532), (2, 0.50725573), (3, 0.070186555), (4, 0.047750182), (5, 0.06744795)]
อยากทราบว่าดิฉันได้ทำงานแล้วได้ยักยอกทรัพย์ของบริษัทกับเพื่อนไปประมาณสองแสนบาทถ้าทางบริษัทจะเอาก่อนเป็นก้อนแล้วที่เหลือให้เราผ่อนชำระถ้าเราไม่มีให้เป็นก้อนแต่เราให้มารดาไปประณีประณอมหนี้โดยการผ่อนชำระจะได้มั้ยถ้าเขาไม่ยอมแล้วแจ้งความตำรวจจะทำยังไงบ้างรบกวนตอบคำถามด้วยนะค

Document ID: 4973, Similarity score: 0.9992378950119019
การตรวจสอบการล้มละลายตรวจสอบได้ที่ศาลล้มละลายหากไม่สามารถดำเนินการได้ด้วยตนเองสามารถให้ทนายคลายทุกข์ดำเนินการแทนได้โทร
Topic distribution for similar document : 
	(Personal Right, 0.17617)
	(Labor, 0.11409)
	(Contract, 0.51518)
	(Sentence, 0.07398)
	(Succession, 0.06002)
	(Family, 0.06057)
Document ID: 2667, Similarity score: 0.9988622665405273
การ้ยืมเงินต้องมีหลักฐานการ้ยืมเงินจึงจะฟ้องร้องได้กรณีของคุณแม้มีเทปบันทึกการสนทนาคงไม่สามารถฟ้องร้องได้
Topic distribution for similar document : 
	(Personal Right, 0.19129)
	(Lab